**Exercicio**

Crea un mapa coroplético mundial atendendo aos datos covid de infeccións nunha data determinada.

Utiliza as seguinte fontes de datos:
- Mapa en formato GeoJSON: https://geojson-maps.ash.ms/
- Datos covid: Narrativa Covid-19 tracking project: https://covid19tracking.narrativa.com/index_en.html 
- Capitais de provincias: CSV: https://github.com/reganjohn/BHI/blob/master/topology/country-capitals.csv

In [ ]:
# Estratexia
# 1. Cargo os datos
# 2. Comprobo cada dataset
# 3. Pinto o mapa

In [ ]:
# MAPA DO MUNDO

# https://geojson-maps.ash.ms/
# Configuro e descargo un mapa en 'Low resolution'
# nome: custom_worldmap_geo.json

# Cargo o ficheiro GeoJSON
# Visualizo os datos

import geopandas as gpd

worldmap = gpd.read_file('../../datasets/custom_worldmap_geo.json')

In [ ]:
worldmap.head(3)

In [ ]:
worldmap.plot()

In [ ]:
# DATOS COVID

# Descargo a través da API os datos de todos os países
# Finalmente creo un dataframe 'covid_paises' cos seguintes datos: id, name, today_confirmed, today_deaths

import requests

data = '2022-03-10'

url = 'https://api.covid19tracking.narrativa.com/api/'+data
response = requests.get(url)

paises = response.json()['dates'][data]['countries']

#print(response.text)


In [ ]:
import pandas as pd

datos_paises = []
for pais in paises:
    id = response.json()['dates'][data]['countries'][pais]['id']
    name = response.json()['dates'][data]['countries'][pais]['name']
    today_confirmed  = response.json()['dates'][data]['countries'][pais]['today_confirmed']
    today_deaths = response.json()['dates'][data]['countries'][pais]['today_deaths']
    datos_paises.append([id,name,today_confirmed,today_deaths])

covid_paises = pd.DataFrame(datos_paises, columns=(['id','name','today_confirmed','today_deaths']))
covid_paises.head()


In [ ]:
# Capitais de países
# Carga das capitais

url_capitais = 'https://raw.githubusercontent.com/reganjohn/BHI/master/topology/country-capitals.csv'
capitais = pd.read_csv(url_capitais,encoding='iso-8859-15',sep=',',on_bad_lines='skip')

# Neste caso só temos que indicar o encoding pero lembra o parámetro 'sep', e tamén 'on_bad_lines', que pode axudar en caso de ficheiros mal formateados


In [ ]:
capitais.head()

In [ ]:
# Finalmente pinto o mapa con todos os elementos
# - Podemos utilizar Circle en lugar de Marker
# - En lugar de popup podemos experimentar co parámetro 'tooltip

import folium

# Podería pasar directamente o ficheiro geojson para pintar o mapa
# Como xa está geojson cargado nun dataframe utilizarei o dataframe
# mapa_paises = '../../datasets/custom_worldmap_geo.json'

m = folium.Map(location=[43, -8.20],zoom_start=2, width=800, height=600)


folium.Choropleth(
    geo_data=worldmap,
    name="choropleth",
    data=covid_paises,
    columns=["name", "today_confirmed"],
    key_on="feature.properties.admin",
    fill_color="YlGn",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Contaxios Covid",
).add_to(m)

for idx, capital in capitais.iterrows():
    folium.Circle([capital.latitude,capital.longitude],tooltip=(f'{capital.capital} ({capital.country})')).add_to(m)

folium.LayerControl().add_to(m)

m

In [ ]:
# Poderiamos investigar cales foron os países que non cruzaron ben os datos e tentar solucionalo a man...
# ou buscar outros datasets que conteñan os mesmos datos

worldmap[worldmap.merge(covid_paises,left_on='admin',right_on='name',how='left').id.isnull()]

In [ ]:
# O ideal sería que os nosos datasets utilizasen algunha codificación internacional, de maneira que non tivésemos que utilizar
# nomes como campos para relacionar táboas.

# Códigos ISO 3166-1  # http://utils.mucattu.com/iso_3166-1.html

# No seguinte repositorio podes encontrar información interesante relacionada co tema:
# https://github.com/datasets/country-codes